In [ ]:
import os


# taking genes from file
source = 'C:/AGlab/IB/opened/'
final = 'C:/AGlab/gene_sorted.txt'

g = open(final, 'w')
g.close()

files = os.listdir(source)
for gff in files:
    with open(source + gff, 'r') as f:
        string = ''
        string += gff.split('_')[2] + ' '
        #string += gff.split('.')[0] + ' '
        for lines in f:
            if not lines.startswith('#'):
                if lines.split()[2] == 'gene':
                    try:
                        if lines.split()[8].split(';')[3].split('=')[1] == 'protein_coding' or \
                            lines.split()[8].split(';')[3].split('=')[1] == 'Gene':
                            None
                        else:
                            #string += lines.split()[8].split(';')[0].split('-')[1].split('_')[1] + ' '
                            string += lines.split()[8].split(';')[3].split('=')[1] + ' '
                    except IndexError:
                        None
        string += '\n'
        g = open(final, 'a')
        if len(string.split()) > 1000:
            g.write(string)
        #print(gff.split('_')[2])
        g.close()

In [ ]:
import pandas as pd


# making table for R vizualization 
file = 'C:/AGlab/gene_sorted.txt'

gene_names = {}
assmeblies_names = []
with open(file, 'r') as f:
    for lines in f:
        main_list = lines.split()
        n = main_list[0]
        assmeblies_names.append(n)
        main_list.pop(0)
        for genes in main_list:
            if genes in gene_names.keys():
                gene_names[genes].append(n)
            else:
                gene_names[genes] = [n]

# for i, v in gene_names.items():
#     if set(v) == 5:
#         print(i, set(v))

In [ ]:
import os


# making statistics for counting filetic patterns
dna = []

source = 'C:/AGlab/IB/opened/'
final = 'C:/AGlab/gene_around.txt'
ended = 'C:/AGlab/around.txt'

g = open(final, 'w')
g.write('name ' + 'start ' + 'end ' + 'strand' + '\n')
g.close()

files = os.listdir(source)
for gff in files:
    for var in dna:
        if var in gff:
            with open(source + gff, 'r') as f:
                with open(final, 'a') as d:
                    for lines in f:
                        if not lines.startswith('#'):
                            if lines.split()[2] == 'gene':
                                d.write(lines.split()[8].split(';')[1].split('=')[1] +
                                        ' ' +
                                        lines.split()[3] + ' ' +
                                        lines.split()[4] + ' ' +
                                        lines.split()[6] + ' ' +
                                        str(var) +
                                        '\n')

with open(final, 'r') as f:
    with open(ended, 'w') as e:
        n = 0
        for lines in f:
            if lines.split()[0] == 'prkT':
                m = 0
                with open(final, 'r') as f1:
                    e.write(lines.split()[4] + '\n')
                    for lines in f1:
                        if n - 10 < m < n + 10:
                            e.write(lines.split()[0] + ' ' +
                                    lines.split()[1] + ' ' +
                                    lines.split()[2] + ' ' +
                                    lines.split()[3] + '\n')
                        m += 1
                e.write('\n')
            n += 1

In [ ]:
import os
from pathlib import Path
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord


# script for working with 16S
path = r'C:\Users\ALEX\Desktop\assemblies'

def get_16s(seq, value):
    result = []
    for rec in SeqIO.parse(seq, 'genbank'):
        for feature in rec.features:
            if value in feature.qualifiers.get('product', []):
                result.append(feature.location.extract(rec.seq))
    return result

# rna_names = get_16s_names(os.path.join(path, assembly))
rrna = ['ribosomal RNA-16S', '16S ribosomal RNA']
rrna_16s_mapping = {}
for rrna_name in rrna:
    for assembly_name in os.listdir(path):
        assembly = os.path.join(path, assembly_name)
#         print(get_16s_names(assembly))
        rrna_16s_mapping[assembly_name] = get_16s(os.path.join(path, assembly), rrna_name)

fasta_path = r'C:\Users\ALEX\Desktop\assemblies\fasta_16s'

with open(os.path.join(fasta_path, f"Bacillus_16s.fasta"), 'a+') as general:
    for assembly in rrna_16s_mapping:
        for i, seq_16s in enumerate(rrna_16s_mapping[assembly]):
            assembly_name = assembly[:-5]
            general.write(f'>{assembly_name}_{i}_Bacillus_16s\n')
            general.write(str(seq_16s) + '\n')